In [1]:
import torch
from zipfile import ZipFile
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import csv

In [6]:
#Extract photos
directory = r"D:\Python\Databases\OpenSARShip2"
files = []

for file in os.listdir(directory):
        if file.endswith('.zip'):
            name = file.split(".")[0]
            files.append(name)


for file in files:
    file = os.path.join(directory, file)
    with ZipFile(file+".zip", 'r') as zip:
        list_files = zip.namelist()
        for zip_file in list_files[:]:
            if zip_file.split("/")[0] != "Patch_Uint8":
                list_files.remove(zip_file)

        for zip_file in list_files:
            zip.extract(zip_file, path=directory)

In [7]:
#Tiff to png

directory = r"D:\Python\Databases\OpenSARShip2\Patch_Uint8"
name = "Visual_Anti-pollution equipment_x20994_y11371_vh.tif"

for file in os.listdir(directory):
    name = file.split(".tif")[0]
    png = os.path.join(directory, name+".png")
    tif = os.path.join(directory, file)
    with Image.open(tif) as img:
        img.save(png)
        os.remove(tif)

#Save to .npz format
directory = r"D:\Python\Databases\OpenSARShip2\Patch_Uint8"

types = []
for file in os.listdir(directory):
    _, type, _=file.split("_", 2)
    types.append(type)

types = list(dict.fromkeys(types))

types = ["Cargo",
"Dredging or underwater ops",
"Fishing",
"High speed craft",
"Law Enforcement",
"Other Type",
"Passenger",
"Tanker",
"Towing",
"Tug"]

image = []
vessel_class = []

for file in os.listdir(directory):
    _, type, _=file.split("_", 2)
    file = os.path.join(directory, file)
    im = Image.open(file)
    single_array = np.array(im)
    try:
        number_class = types.index(type)
        image.append(single_array)
        vessel_class.append(number_class)
    except:
        os.remove(file)

names = os.listdir(directory)  
header = ["filename", "vessel_type"]

csv_file = directory = r"D:\Python\Databases\OpenSARShip2\Patch_Uint8\dataset.csv"

with open(csv_file, "w", encoding="UTF8") as f:
    writer = csv.writer(f)

    writer.writerow(header)
    for name, type in zip(names, vessel_class):
        row = (name, type)
        writer.writerow(row)


#np.savez("ship_dataset.npz", image=image, vessel_class=vessel_class, dtype=object) # save all in one file

In [ ]:
#Create balanced train and test datasets

X = df["filename"]
y = df["vessel_type"]

from sklearn.model_selection import StratifiedShuffleSplit
sss=StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=9)


for train_index, test_index in sss.split(X, y):
    train_index= train_index
    test_index = test_index
    

init_directory = r"D:\Python\Databases\Patch_Uint8"
final_directory = r"D:\Python\Databases\Patch_Uint8\test"

for index in test_index:
    name = df["filename"][index]

    init_filename = os.path.join(init_directory, name)
    final_filename =  os.path.join(final_directory, name)
    os.rename(init_filename, final_filename )


In [67]:
#Resize images
def resize_image(filename):
    new_size = 180
    delta= int(new_size/2)
    with Image.open(filename) as img:
        img.load()

    width, height = img.size
    
    if height>new_size:
        new_img = img.crop((width/2 - delta, height/2 - delta, width/2 + delta, height/2 + delta))
        new_img.save(filename)  
    elif height<new_size:

        new_img = Image.new(img.mode, (new_size, new_size))

        new_img.paste(img, (delta - int(width/2), delta - int(height/2)))
        new_img.save(filename)

from PIL import Image
init_directory = r"D:\Python\Databases\Patch_Uint8\test"

files = os.listdir(init_directory)


for file in files:

    filename = os.path.join(init_directory, file)

    resize_image(filename)

In [18]:
#Read npz database
data = np.load(r"D:\Python\Databases\ship_dataset\ship_dataset.npz", allow_pickle=True) 
files = data.files 

print('Shape - "Image" item: ' + str(data['image'].shape))
print('Shape - "Label" item: ' + str(data['vessel_class'].shape))

index = 3454
plt.imshow(data['image'][index], cmap='Greys')
print('Ship class: ' + str(data['vessel_class'][index]))

for i in range(10):
    veces = np.count_nonzero(data['vessel_class'] == i)
    print(f"La clase {i} ha aparecido {veces} veces")

3852


In [2]:
class vesselDataset(Dataset):
    def __init__(self):
        pass
    def __getitem__(self,index):
        pass

    def __len__(self):
        pass

In [13]:
#Clean data
directory = "D:\Python\Databases\ship_dataset_reduced"

files = os.listdir(directory)
for file in files:
        if file.endswith('.jpg'):
            files.remove(file)

for file in files:
    filename = os.path.join(directory, file)
    with open(filename, 'r') as fp:
        if len(fp.readlines())>1:
            print(file)

for file in files:
    name = file.split(".")[0]
    txt = os.path.join(directory, name+".txt")
    picture = os.path.join(directory, name+".jpg")
    os.remove(txt)
    os.remove(picture)

directory = "D:\Python\Databases\ship_dataset_reduced"
files = []
for file in os.listdir(directory):
        if file.endswith('.jpg'):
            name = file.split(".")[0]
            files.append(name)

train_folder = os.path.join(directory, "train")
test_folder = os.path.join(directory, "test")

os.mkdir(train_folder)
os.mkdir(test_folder)

for file in os.listdir(directory):
    if file.endswith('.jpg'):
        name = file.split(".")[0]
        files.append(name)

vessel_classes = []
normalized_ship_widths = []
normalized_ship_lenghts = [] 

for file in files:
    txt = os.path.join(directory, file)+".txt"
    with open(txt, 'r') as fp:
        data = fp.readline().strip().split(" ")
        vessel_classes.append(data[0])
        normalized_ship_widths.append(data[3])
        normalized_ship_lenghts.append(data[4])
    